In [1]:
from pathlib import Path 
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

import torch
import torch.nn as nn

import pandas as pd 
import numpy as np 
import re


In [2]:
path = Path('../data/')


dataverse_path =  path / 'dataverse_files'


partitions_list = [


    'partition3_instances',

    
    ]



In [ ]:
def parse_partition(partitions_list, dataverse_path, max_workers=3):

    feature_list_summaries_original = [ 'Timestamp', 'TOTUSJH', 'TOTBSQ', 'TOTPOT', 'TOTUSJZ', 'ABSNJZH',
       'SAVNCPP', 'USFLUX', 'TOTFZ', 'MEANPOT', 'EPSZ', 'MEANSHR', 'SHRGT45',
       'MEANGAM', 'MEANGBT', 'MEANGBZ', 'MEANGBH', 'MEANJZH', 'TOTFY',
       'MEANJZD', 'MEANALP', 'TOTFX', 'EPSY', 'EPSX', 'R_VALUE']
    
    feature_list_summaries = [ 'TOTUSJH', 'TOTBSQ', 'TOTPOT', 'TOTUSJZ', 'ABSNJZH',
       'R_VALUE']
    
    # First pass: collect all files to show total progress
    all_files = []
    for partition_inst in partitions_list:
        partition_inst_path = dataverse_path / partition_inst
        partition_num = partition_inst.replace('_instances', '')
        partition_folder_path = partition_inst_path / partition_num
        
        for folder in ['FL', 'NF']:
            folder_path = partition_folder_path / folder
            if folder_path.exists() and folder_path.is_dir():
                for file in folder_path.glob('*.csv'):
                    all_files.append((file, folder, partition_num))
    

    def parse_file(file_info):

        file, class_label, partition_num = file_info
        filename = file.stem
        
        df = pd.read_csv(
            file,
            sep="\t",
            na_values=["", " ", "NA", "NaN", "null", "None", "inf", "-inf"]
        )

        #ts = df['Timestamp'].copy()

        df = df[feature_list_summaries].copy()

        # force numeric
        df = df.apply(pd.to_numeric, errors="coerce")

        # treat inf as missing
        df = df.replace([np.inf, -np.inf], np.nan)

        # interpolate + fill
        df = (
            df
            .interpolate(method="linear", axis=0, limit_direction="both")
            .ffill()
            .bfill()
        )

        # if any columns were entirely NaN, set them to 0
        all_nan_cols = df.columns[df.isna().all()]
        if len(all_nan_cols) > 0:
            df[all_nan_cols] = 0.0

        # final hard assert
        if df.isna().any().any():
            bad = df.columns[df.isna().any()].tolist()
            raise ValueError(f"Still have NaNs after fill in columns: {bad}")



        # Parse filename - handle both FL and NF cases
        if '@' in filename:
            # Format: M1.0@265_Primary_ar115_s2010-08-06T08:36:00_e2010-08-06T20:24:00
            flare_part, rest = filename.split('@', 1)
            flare_class = flare_part.strip()[0]

            flare_id_part, location_part = rest.split('_', 1)
            flare_id = flare_id_part.strip()

            ar_match = re.search(r'_ar(\d+)', filename)
            if ar_match:
                active_region = ar_match.group(1)


        else:
            # NF case - parse similar format without flare ID
            flare_class = 'NF'
            flare_id = 0

            ar_match = re.search(r'_ar(\d+)', filename)
            if ar_match:
                active_region = ar_match.group(1)


        return {"id": flare_id, "values": df.values.astype('float32'), "class": flare_class, "active_region" : active_region}
    
    # call threads here ( pls run this threaded )
    all_results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for result in tqdm(executor.map(parse_file, all_files), total=len(all_files), desc="Parsing files"):
            all_results.append(result)
    

    results_df = pd.DataFrame(all_results)
    return results_df 

In [4]:
result = parse_partition(partitions_list, dataverse_path)
print(f"\nTotal files parsed: {len(result)}")
result

Parsing files: 100%|██████████| 42510/42510 [02:58<00:00, 238.49it/s]



Total files parsed: 42510


,id,values,class,active_region
0,6861,"[[2380.2966, 2.5360703e+10, 4.931398e+23, 3.19...",M,3311
1,6861,"[[2311.8174, 2.5015935e+10, 4.9003286e+23, 3.1...",M,3311
2,6861,"[[2474.5442, 2.4952439e+10, 4.588807e+23, 3.34...",M,3311
3,6861,"[[2262.4639, 2.4312164e+10, 4.1395333e+23, 3.0...",M,3311
4,6861,"[[2149.771, 2.3488825e+10, 4.434565e+23, 3.028...",M,3311
...,...,...,...,...
42505,0,"[[919.10614, 8.979812e+09, 6.3049643e+22, 1.92...",NF,4197
42506,0,"[[908.9685, 8.82262e+09, 6.10126e+22, 1.865239...",NF,4197
42507,0,"[[973.226, 9.030366e+09, 6.6438264e+22, 1.8894...",NF,4197
42508,0,"[[1065.6189, 9.840453e+09, 7.516114e+22, 2.112...",NF,4197


In [5]:
print(result.groupby("class").size())
print(result['values'][0].shape)

class
B       685
C      5639
M      1288
NF    34762
X       136
dtype: int64
(60, 6)


In [6]:
x_and_y = result.copy()
convert_dict = {'NF':0, 'B':1, 'C':1, 'M':2, 'X':2}
x_and_y['class'] = x_and_y['class'].map(convert_dict)
x_and_y

,id,values,class,active_region
0,6861,"[[2380.2966, 2.5360703e+10, 4.931398e+23, 3.19...",2,3311
1,6861,"[[2311.8174, 2.5015935e+10, 4.9003286e+23, 3.1...",2,3311
2,6861,"[[2474.5442, 2.4952439e+10, 4.588807e+23, 3.34...",2,3311
3,6861,"[[2262.4639, 2.4312164e+10, 4.1395333e+23, 3.0...",2,3311
4,6861,"[[2149.771, 2.3488825e+10, 4.434565e+23, 3.028...",2,3311
...,...,...,...,...
42505,0,"[[919.10614, 8.979812e+09, 6.3049643e+22, 1.92...",0,4197
42506,0,"[[908.9685, 8.82262e+09, 6.10126e+22, 1.865239...",0,4197
42507,0,"[[973.226, 9.030366e+09, 6.6438264e+22, 1.8894...",0,4197
42508,0,"[[1065.6189, 9.840453e+09, 7.516114e+22, 2.112...",0,4197


In [11]:
def add_gaussian_noise(X_train,y_train,minority=1,samples=30000,sigma=0.01, random_state=42):

    X_train = pd.DataFrame(X_train).copy()
    y_train = pd.Series(y_train).copy()


    minority_mask = (y_train == minority) # query T F for row in target minority 
    X_minority = X_train[minority_mask].copy()

    Sampled_X = X_minority.sample(n=samples,replace=True,random_state=random_state)

    #ensure numerica columns here 


    X_num = Sampled_X.select_dtypes(include=[np.number])
    feature_std = X_num.std(axis=0).replace(0,1e-16)

    np.random.seed(random_state)
    noise = np.random.normal(loc=0,scale=sigma,size=X_num.shape) * feature_std.values

    X_noise_num = X_num + noise

    X_noise = Sampled_X.copy()
    X_noise[X_num.columns] = X_noise_num

    y_noise = pd.Series([minority] * samples,index=X_noise.index)

    X_aug = pd.concat([X_train,X_noise], ignore_index=True)
    Y_aug = pd.concat([y_train,y_noise.reset_index(drop=True)],ignore_index=True)

    return X_aug,Y_aug

In [ ]:
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold, StratifiedKFold
from sklearn.utils import compute_class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import torch.utils.data as data
import torch.optim as optim

#Basic LSTM Model with a dropout layer
class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=6, hidden_size=24, batch_first=True, num_layers=1)
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(24, 3)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.dropout(x)
        x = self.linear(x)

sgkf = StratifiedGroupKFold(n_splits=3)

X = x_and_y['values']
y = x_and_y['class']
groups = x_and_y["active_region"].astype('int')

for i, (train_index, test_index) in enumerate(sgkf.split(X, y, groups)):
    print(f"Fold {i+1}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    

    #Split into training and testing data
    X_train, X_test = X.iloc[train_index].copy(), X.iloc[test_index].copy()
    y_train, y_test = y.iloc[train_index].copy(), y.iloc[test_index].copy()

    print("Train distribution:", np.bincount(y_train))
    print("Test distribution:", np.bincount(y_test))

    #add gaussian noise
    print("Before:", X_train.shape, y_train.shape)
    X_train, y_train = add_gaussian_noise(X_train, y_train, samples=2000, minority=2)
    X_train, y_train = add_gaussian_noise(X_train, y_train, samples=2000, minority=1)
    X_train = X_train.values.ravel() 
    print("After:", X_train.shape, y_train.shape)

    #Stack train and test values into two continuous arrays
    X_train, X_test = np.stack(X_train), np.stack(X_test.to_numpy())

    #Reshaping arrays for scaling
    train_shape = X_train.shape
    test_shape = X_test.shape
    X_train_reshape = X_train.reshape(-1, train_shape[2])
    X_test_reshape = X_test.reshape(-1, test_shape[2])

    #Scale and reshape back.
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_reshape)
    X_test_scaled = scaler.transform(X_test_reshape)

    X_train_scaled = X_train_scaled.reshape(train_shape)
    X_test_scaled = X_test_scaled.reshape(test_shape)
    
    #Convert data to tensors before inputting into model
    X_train = torch.tensor(X_train_scaled, dtype=torch.float32)
    X_test = torch.tensor(X_test_scaled, dtype=torch.float32)

    buh = y_train.to_numpy()
    y_train = torch.tensor(y_train.to_numpy(), dtype=torch.long)
    y_test = torch.tensor(y_test.to_numpy(), dtype=torch.long)

    print("Train distribution:", torch.bincount(y_train))
    print("Test distribution:", torch.bincount(y_test))

    #Initialising the model, optimiser, loss function, and loader before training loop
    model = LSTMModel()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

    class_weights = compute_class_weight('balanced', classes=np.unique(buh), y=buh)
    class_weights = torch.tensor(class_weights, dtype=torch.float32)

    loss_fn = nn.CrossEntropyLoss(weight=class_weights)

    loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=64)

    for epoch in range(50):
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in loader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()

        model.eval()
        val_loss, acc, f1 = 0.0, 0.0, 0.0
        all_preds = []
        all_true = []

        with torch.no_grad():
            y_pred = model(X_test)
            loss = loss_fn(y_pred, y_test)

            all_preds.extend(torch.argmax(y_pred, dim=1).numpy())
            all_true.extend(y_test.numpy())

            acc = (torch.argmax(y_pred, dim=1) == y_test).float().mean()
            acc = float(acc)
            f1 = f1_score(y_test.numpy(), torch.argmax(y_pred, dim=1).numpy(), average="macro")
            val_loss += loss.item()
            print(torch.bincount(torch.argmax(y_pred, dim=1)))

        print("Epoch %d: Training Loss: %f,  Validation Loss: %f, Accuracy: %f, Macro-f1: %f" % (epoch + 1, train_loss / len(loader), val_loss , acc, f1))
    print(classification_report(all_true, all_preds))
    #print(confusion_matrix(all_true, all_preds))


Fold 1:
  Train: index=[    0     1     2 ... 42507 42508 42509]
  Test:  index=[    9    10    11 ... 42493 42494 42495]
Train distribution: [23174  4217   956]
Test distribution: [11588  2107   468]
Before: (28347,) (28347,)
After: (32347,) (32347,)
(1940820, 6) (32347,)
Train distribution: tensor([23174,  6217,  2956])
Test distribution: tensor([11588,  2107,   468])
tensor([9496, 3333, 1334])
Epoch 1: Training Loss: 0.685295,  Validation Loss: 0.613175, Accuracy: 0.752877, Macro-f1: 0.545626
tensor([9765, 3747,  651])
Epoch 2: Training Loss: 0.597728,  Validation Loss: 0.611571, Accuracy: 0.791640, Macro-f1: 0.574493
tensor([9705, 3202, 1256])
Epoch 3: Training Loss: 0.567472,  Validation Loss: 0.630464, Accuracy: 0.767281, Macro-f1: 0.555381
tensor([9294, 3830, 1039])
Epoch 4: Training Loss: 0.563306,  Validation Loss: 0.643601, Accuracy: 0.757325, Macro-f1: 0.556503
tensor([9372, 3568, 1223])
Epoch 5: Training Loss: 0.564110,  Validation Loss: 0.622985, Accuracy: 0.752242, Macro-